<a href="https://colab.research.google.com/github/niravshah241/homogenization/blob/main/homogenization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import zipfile
try:
  import google.colab
except ImportError:
  from dolfin import *
else:
  try:
    from dolfin import *
  except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    from dolfin import *
%matplotlib inline

--2022-10-30 14:37:01--  https://fem-on-colab.github.io/releases/fenics-install.sh
Resolving fem-on-colab.github.io (fem-on-colab.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to fem-on-colab.github.io (fem-on-colab.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2864 (2.8K) [application/x-sh]
Saving to: ‘/tmp/fenics-install.sh’

/tmp/fenics-install 100%[===================>]   2.80K  --.-KB/s    in 0s      

2022-10-30 14:37:01 (42.5 MB/s) - ‘/tmp/fenics-install.sh’ saved [2864/2864]

+ SHARE_PREFIX=/usr/local/share/fem-on-colab
+ FENICS_INSTALLED=/usr/local/share/fem-on-colab/fenics.installed
+ [[ ! -f /usr/local/share/fem-on-colab/fenics.installed ]]
+ PYBIND11_INSTALL_SCRIPT_PATH=https://github.com/fem-on-colab/fem-on-colab.github.io/raw/8aad56e/releases/pybind11-install.sh
+ [[ https://github.com/fem-on-colab/fem-on-colab.github.io/raw/8aad56e/releases/pybind11-install.sh == http* ]]
+ PYBIND11_

In [3]:
#1. Domain discretisation
!wget "https://github.com/niravshah241/homogenization/archive/refs/heads/main.zip" -O "/tmp/github_files.zip"

zip_ref = zipfile.ZipFile("/tmp/github_files.zip", 'r')
zip_ref.extractall("/tmp")
zip_ref.close()

mesh = Mesh("/tmp/homogenization-main/mesh_data/mesh.xml")
subdomains = MeshFunction("size_t", mesh, "/tmp/homogenization-main/mesh_data/mesh_physical_region.xml")
boundaries = MeshFunction("size_t", mesh, "/tmp/homogenization-main/mesh_data/mesh_facet_region.xml")

dx = Measure("dx")(subdomain_data=subdomains)
ds = Measure("ds")(subdomain_data=boundaries)

dx_left = dx(1) #Subdomain 1
dx_right = dx(2) #Subdomain 2

ds_left = ds(1) #Left boundary
ds_right = ds(5) #Right boundary
ds_top = ds(2) + ds(4) #Top boundary
ds_botom = ds(3) + ds(6) #Bottom boundary

n = FacetNormal(mesh)

--2022-10-30 14:37:23--  https://github.com/niravshah241/homogenization/archive/refs/heads/main.zip
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/niravshah241/homogenization/zip/refs/heads/main [following]
--2022-10-30 14:37:23--  https://codeload.github.com/niravshah241/homogenization/zip/refs/heads/main
Resolving codeload.github.com (codeload.github.com)... 140.82.113.9
Connecting to codeload.github.com (codeload.github.com)|140.82.113.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘/tmp/github_files.zip’

/tmp/github_files.z     [ <=>                ]  27.42K  --.-KB/s    in 0.004s  

2022-10-30 14:37:23 (6.25 MB/s) - ‘/tmp/github_files.zip’ saved [28075]



In [ ]:
# 2. Create Finite Element space (Lagrange P1)
VT = FunctionSpace(mesh,"CG",1) # Space for temperature field
T_, psi = TrialFunction(VT), TestFunction(VT)
T = Function(VT,name="Temperature") # Temperature field
T_eq = Function(VT,name="Temperature_Equivalent") # Temperature field with equaivalent conductivity

if VT.ufl_element().degree() == 1:
  VQ = VectorFunctionSpace(mesh,"CG",VT.ufl_element().degree()) # Space for heat flux
else:
  VQ = VectorFunctionSpace(mesh,"CG",VT.ufl_element().degree()-1) # Space for heat flux
q_, psi_q = TrialFunction(VQ), TestFunction(VQ)
q = Function(VQ,name="Heat flux")

#dofs = VT.dofmap().dofs()
#print(dofs)

In [ ]:
# 3. Material and boundary properties, Source term
k_left = 8. # Thermal conductivity left subdomain
k_right = 4. # Thermal conductivity right subdomain
T_left = Constant(1800.) # Temperature on left boundary
T_right = Constant(300.) # Temperature on right boundary

In [ ]:
# 4. Compute heat flux

def compute_heat_flux(temp_field):
  '''
  Compute the heat flux at given temperature field
  Input:
  temp_field: Function over Functionspace
  Output:
  q: Function over VectorFunctionSpace
  '''
  a_q = inner(q_, psi_q) * dx_left + inner(q_, psi_q) * dx_right
  l_q = inner(-k_left * grad(temp_field), psi_q) * dx_left + inner(-k_right * grad(temp_field), psi_q) * dx_right
  solve(a_q == l_q, q)
  return q

In [ ]:
# 5. Weak formulation
a_T = inner(k_left*grad(T_),grad(psi)) * dx_left + inner(k_right*grad(T_),grad(psi)) * dx_right
l_T = inner(Constant(0.),psi) * dx

bcs = [DirichletBC(VT,T_left,boundaries,1),DirichletBC(VT,T_right,boundaries,5)]

solve(a_T==l_T,T,bcs)

In [ ]:
# 6. Post processing

xdmf_file = XDMFFile("/tmp/homogenization-main/solution_field/temperature.xdmf") #Path to save output files in google drive
xdmf_file.parameters["flush_output"] = True
xdmf_file.parameters["functions_share_mesh"] = True
xdmf_file.parameters["rewrite_function_mesh"] = False
xdmf_file.write(T,0)
gradT = project(grad(T),VQ)
gradT.rename("Temperature gradient","some label")
xdmf_file.write(gradT,0)
xdmf_file.write(compute_heat_flux(T),0)

In [ ]:
# 7. Compute equivalent conductivity and solution fields
k_eq = -assemble(compute_heat_flux(T)[0]*dx) / assemble(T.dx(0)*dx)
print(k_eq)

a_T = inner(k_eq*grad(T_),grad(psi)) * dx
l_T = inner(Constant(0.),psi) * dx

bcs = [DirichletBC(VT,T_left,boundaries,1),DirichletBC(VT,T_right,boundaries,5)]

solve(a_T==l_T,T_eq,bcs)

plot(T_eq)
plt.show()

In [ ]:
# 8. Post processing
xdmf_file = XDMFFile("/tmp/homogenization-main/solution_field/temperature_equivalent.xdmf") #Path to save output files in google drive
xdmf_file.parameters["flush_output"] = True
xdmf_file.parameters["functions_share_mesh"] = True
xdmf_file.parameters["rewrite_function_mesh"] = False
xdmf_file.write(T_eq,0)
gradTeq = project(grad(T_eq),VQ)
gradTeq.rename("Equivalent temperature gradient","some label2")
xdmf_file.write(gradTeq,0)
xdmf_file.write(compute_heat_flux(T_eq),0)

In [ ]:
# 9. Error assessment and post processing of error
error = norm(project(T-T_eq,VT),"L2")/norm(T,"L2")
print("Error: ",error)

xdmf_file = XDMFFile("/tmp/homogenization-main/solution_field/spatial_error.xdmf") #Path to save output files in google drive
xdmf_file.parameters["flush_output"] = True
xdmf_file.parameters["functions_share_mesh"] = True
xdmf_file.parameters["rewrite_function_mesh"] = False
xdmf_file.write(project(T-T_eq,VT),0)
xdmf_file.write(project(grad(T-T_eq),VQ),0)
xdmf_file.write(project(compute_heat_flux(T-T_eq),VQ),0)

In [ ]:
# 10. Save values in numpy format
np.save("/tmp/homogenization-main/solution_field/mesh_x_coordinates.npy",mesh.coordinates()[:,0])
np.save("/tmp/homogenization-main/solution_field/mesh_y_coordinates.npy",mesh.coordinates()[:,1])
np.save("/tmp/homogenization-main/solution_field/temperature_at_nodes.npy",T.vector().get_local())
np.save("/tmp/homogenization-main/solution_field/equivalent_temperature_at_nodes.npy",T_eq.vector().get_local())
np.save("/tmp/homogenization-main/solution_field/temperature_gradient_at_nodes.npy",gradT.sub(0).vector().get_local())
np.save("/tmp/homogenization-main/solution_field/equivalent_temperature_gradient_at_nodes.npy",gradTeq.sub(0).vector().get_local())